**Get Data BODY and split words**

In [1]:
import re

def delTrashChar(string, trash):
    for t in trash:
        string = string.replace(t, ' ')
    return re.sub(' +', ' ', string)

In [2]:
data = sc.wholeTextFiles('data/reut2-*')
news_articles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                    .map(lambda x:delTrashChar(x, '~!@#$%^&*()_+`-=/\\,.<>\'\";:{}[]\n\t|'))

N = news_articles.count()
print(N, 'News Articles(N)')

19043 News Articles(N)


**(1) Given the Reuters-21578 dataset, please calculate the term frequencies, and output the representation of the document contents as a term-document count matrix.**  
**Output Format  
term-document matrix: A MxN matrix: with rows as term frequencies and columns as documents (N=21,578)**

In [3]:
distinct_words = news_articles.flatMap(lambda x:x.split(' ')).distinct()

M = distinct_words.count()
print(M, 'Distinct Words(M)')

48447 Distinct Words(M)


In [4]:
from pyspark.mllib.linalg.distributed import RowMatrix

news_articles_list = news_articles.map(lambda x:x.split(' ')).collect()
term_document_count_matrix = RowMatrix(distinct_words.map(lambda x:[article.count(x) for article in news_articles_list]))

In [5]:
term_document_count_matrix.rows.coalesce(1).saveAsTextFile('tmp')

!mv tmp/part-00000 result/term_document_count_matrix.txt
!rm -rf tmp

**(2) Implement matrix multiplication by MapReduce. Your program should be able to output the result in appropriate dimensions.**  
**Output Format  
result of matrix multiplication: An MxR matrix: (MxN) * (NxR)**

**(3) Given the term-document matrix in (1), compute the SVD decomposition of the matrix using MapReduce. Output the resulting eigenvalues and eigenvectors.**  
**Output Format  
eigenpairs: Eigenvalues sorted in descending order, and their corresponding eigenvectors**

>**I do the Q2 Q3 at same time, I compute top-10 eigenvalues and use these eigenvectors to multiply with term-document count matrix**

In [6]:
R = 10

In [7]:
svd = term_document_count_matrix.computeSVD(R)

In [8]:
matrixMultiply = term_document_count_matrix.multiply(svd.V)

In [9]:
matrixMultiply.rows.coalesce(1).saveAsTextFile('tmp')

!mv tmp/part-00000 result/matrixMultiply.txt
!rm -rf tmp

In [10]:
V = svd.V.toArray()
s = svd.s.toArray()
with open('result/svd.txt', 'w') as result:
    for i in range(R):
        result.write(str(s[i]) + '\n' + str(V[i]) + '\n\n')